## **Importing all dependencies**

In [7]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mae, mse
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam

## **Changing the directory**

In [6]:
os.chdir(r"filterseps")

## **Defining the model**

In [ ]:
model = Sequential(
    [
        # LSTM(256, activation="relu", input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
        LSTM(128, activation="relu", return_sequences=True),
        LSTM(64, activation="relu", return_sequences=True),
        LSTM(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation="relu"),
        Dense(1)
    ]
)

## **Prediction Pipeline - For Loop Method**

In [ ]:
predictionDict = {}
for k in os.listdir():
    print(f"File No. {k[:-4]}\n")
    df = pd.read_csv(k)
    df = df.drop(['Symbol'], axis=1)
    test = np.array([df[int(0.15*(df.shape[0]-1)):].values])
    df = df[:int(0.15*(df.shape[0]-1))]
    # Making the dataset for each dataframe
    X, y = [], []
    slidingWindow = 1
    completeSampleSize = 50

    for i in range(0, df.shape[0]-completeSampleSize, slidingWindow):
        X.append(df.iloc[i:i+30, :].values)
        y.append(df.iloc[i+30:i+50, 0].values)

    X = np.array(X)
    y = np.array(y)

    print(test)

    print(f"X ki shape: {X.shape}\t\t y ki shape: {y.shape}")
    # Training
    model.compile(loss='mae', optimizer=Adam(0.001))
    model.fit(X, y, epochs=50, validation_split=0.15, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)])
    predictions = model.predict(test)
    predictionDict[k] = predictions
    print(predictions)
    print('-'*50)

## **Calculating the %age increase/decrease**

In [ ]:
differenceDict = {}
for k in os.listdir():
    df = pd.read_csv(k)
    originalValue = df.loc[0,'Price']
    newValue = predictionDict[k]

    change = ((newValue-originalValue)*100)/originalValue

    differenceDict[k] = change

changeDataFrame = pd.DataFrame(differenceDict)